In [40]:
#car follow the path
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
#astar()
client = carla.Client('localhost', 2000)
world = client.get_world()
#world = client.load_world('Town05')
map=world.get_map()


In [35]:
#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

spawn_points = world.get_map().get_spawn_points()
spawn_point = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*model*')

start_point = spawn_point[0]
#start_point.location.y=start_point.location.y+10
end_point1 = spawn_points[0]
end_point1.location.y=end_point1.location.y+110
end_point2=map.get_waypoint(end_point1.location).get_left_lane().transform
#end_point3=map.get_waypoint(end_point2.location).next_until_lane_end(1)[-1].transform
end_point3=map.get_waypoint(carla.Location(x=27.529999732971191,y=190.19000244140625)).transform

ws1=grp.trace_route(start_point.location,end_point1.location)
ws2=grp.trace_route(end_point2.location,end_point3.location)
ws1.extend(ws2)

for w in ws1:
    world.debug.draw_string(w[0].transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)

world.debug.draw_string(start_point.location, '#', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=60.0,
            persistent_lines=True)

world.debug.draw_string(end_point2.location, '#', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=60.0,
            persistent_lines=True)

world.debug.draw_string(end_point3.location, '#', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=60.0,
            persistent_lines=True)

vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False

start=time.time()
while time.time()-start<14:
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)

    estimated_throttle = maintain_speed(speed)
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=0))
    cv2.imshow('RGB Camera',image)

while True:
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=-1))
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('RGB Camera',image)

    if(vehicle.get_transform().rotation.yaw)<=10:
        while(vehicle.get_transform().rotation.yaw)>=-80:
            if cv2.waitKey(1) == ord('q'):
                quit = True
                vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
                break
            image = camera_data['image']
            vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=1,reverse=True))
            v = vehicle.get_velocity()
            speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
            image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
            cv2.imshow('RGB Camera',image)

        break
  
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(ws2)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(ws2) and vehicle.get_transform().location.distance(ws2[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,ws2[curr_wp][0])
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)

cv2.destroyAllWindows()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy() 


In [33]:
cv2.destroyAllWindows()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [2]:
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

ws1=grp.trace_route(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=grp.trace_route(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
wa=map.get_waypoint(carla.Location(x=100,y=103,z=0))
ws3=grp.trace_route(carla.Location(x=193,y=125,z=0),wa.transform.location)
ws4=grp.trace_route(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=grp.trace_route(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=grp.trace_route(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=grp.trace_route(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=grp.trace_route(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
wa=map.get_waypoint(carla.Location(x=165,y=235,z=0))
ws9=grp.trace_route(carla.Location(x=135,y=210,z=0),wa.transform.location)
ws10=grp.trace_route(carla.Location(x=165,y=235,z=0),carla.Location(x=40,y=290,z=0))
ws11=grp.trace_route(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=grp.trace_route(carla.Location(x=95,y=305,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w[0].transform.location, '^', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=255), life_time=6.0,
            persistent_lines=True)

# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

start_point = route[0][0].transform
start_point.location.z=0.5

vehicle_bp = world.get_blueprint_library().filter('*mini*')

vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(route)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,route[curr_wp][0])
    image = cv2.putText(image, 'Steering angle: '+str(round(predicted_angle,3)), org, font, fontScale, color, thickness, cv2.LINE_AA)
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Next wp: '+str(curr_wp), org3, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)
    

#clean up
cv2.destroyAllWindows()
camera.stop()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [71]:
w=map.get_waypoint(carla.Location(x=145.529999732971191,y=30.19000244140625)).transform.location
world.debug.draw_string(w, '#', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=6.0,
        persistent_lines=True)

In [109]:
#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

spawn_points = world.get_map().get_spawn_points()
spawn_point = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*model*')
start_point=spawn_points[0]
w=map.get_waypoint(carla.Location(x=120.529999732971191,y=-0.19000244140625,z=0.5)).transform
start_point.location.x=w.location.x
start_point.location.y=w.location.y
start_point.rotation.yaw=w.rotation.yaw
end_point1=map.get_waypoint(carla.Location(x=142.529999732971191,y=-0.19000244140625,z=0.5)).transform
end_point2=map.get_waypoint(carla.Location(x=145.529999732971191,y=30.19000244140625,z=0.5)).transform

ws1=get_path(start_point.location,end_point1.location)
ws2=get_path(end_point1.location,end_point2.location)
#ws1.extend(ws2)

for w in ws1:
    world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
for w in ws2:
    world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)


vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False
curr_wp = 7 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(ws1)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(ws1)-1 and vehicle.get_transform().location.distance(ws1[curr_wp].transform.location)<7:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,ws1[curr_wp])
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)

print("done")
start=time.time()
while time.time()-start<10:
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Stop for: '+str(int(11-time.time()+start)), org, font, fontScale, (0,0,255), thickness, cv2.LINE_AA)

    estimated_throttle = maintain_speed(speed)
    vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0,brake=1))
    cv2.imshow('RGB Camera',image)

curr_wp = 2 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
print("done")

while curr_wp<len(ws2)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(ws2) and vehicle.get_transform().location.distance(ws2[curr_wp].transform.location)<2:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,ws2[curr_wp])
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)
print("done")
cv2.destroyAllWindows()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy() 


done
done
done


In [102]:
cv2.destroyAllWindows()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy() 


In [100]:
world.debug.draw_string(start_point.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
world.debug.draw_string(end_point1.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
ws1=get_path(start_point.location,end_point1.location)
for w in ws2:
    world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)

In [99]:
print(len(ws1))
print(ws2)

24
[<carla.libcarla.Waypoint object at 0x7f28fe3303c0>, <carla.libcarla.Waypoint object at 0x7f28fe6aff90>, <carla.libcarla.Waypoint object at 0x7f28fe6aff20>, <carla.libcarla.Waypoint object at 0x7f28fe6b1120>, <carla.libcarla.Waypoint object at 0x7f28fe3ab120>, <carla.libcarla.Waypoint object at 0x7f28fe3ab270>, <carla.libcarla.Waypoint object at 0x7f28fe3abc10>, <carla.libcarla.Waypoint object at 0x7f28fe3ab0b0>, <carla.libcarla.Waypoint object at 0x7f28fe3abd60>, <carla.libcarla.Waypoint object at 0x7f28fe3abdd0>, <carla.libcarla.Waypoint object at 0x7f28fe3abe40>, <carla.libcarla.Waypoint object at 0x7f28fe3abeb0>, <carla.libcarla.Waypoint object at 0x7f28fe3abf20>, <carla.libcarla.Waypoint object at 0x7f28fe3abf90>, <carla.libcarla.Waypoint object at 0x7f28fe330040>, <carla.libcarla.Waypoint object at 0x7f28fe3300b0>, <carla.libcarla.Waypoint object at 0x7f28fe330120>, <carla.libcarla.Waypoint object at 0x7f28fe330190>, <carla.libcarla.Waypoint object at 0x7f28fe330200>, <carla.l

In [87]:
waypoint_list = map.generate_waypoints(1.0)
for waypoint in waypoint_list:
    world.debug.draw_string(waypoint.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=255, b=0), life_time=60.0,
        persistent_lines=True)

In [89]:
#global planner without u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

    rout=[]
    if((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):
        (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
        g = len(wa_start.next_until_lane_end(1))
        f=g+dist([x1,y1],[x2,y2])
        open = [[f,g,dist([x1,y1],[x2,y2]), x1, y1]]
        found = False 
        f_distance={node:float('inf') for node in graph}
        f_distance[n2_start]=f
    
        came_from={node:None for node in graph}
        came_from[n2_start]=n2_start
        #came_from={'s':None}
        #came_from={'su':None}
        #came_from={'n':None}
        #came_from={'nu':None}
        #came_from[n2_start]='s'
        
        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                x = next[3]
                y = next[4]
                g = next[1]
                for i in graph[id_map[(x,y)]]:
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[id_map[(x,y)]][i].next_until_lane_end(1))+g
                    f2=g2+dist([xf,yf],[x2,y2])
                    if came_from[i]==None:
                        open.append([f2,g2,dist([xf,yf],[x2,y2]), xf, yf])
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=id_map[(x,y)]
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            ws=map.get_waypoint(carla.Location(x=wa_start.transform.location.x,y=wa_start.transform.location.y,z=0)).next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout
